# Deep Learning for addition

In [64]:
# numpy
import numpy as np

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset

# scikit
from sklearn.model_selection import train_test_split

# visualization

## Generate Data

In [61]:
# generate 500 samples between -10 & + 10 as first input
# generate 500 samples between -10 & + 10 as second input
# y is the sum of first and second sample
nPerClust = 10
a = np.random.randint(-10,10,nPerClust)
b = np.random.randint(-10,10,nPerClust)
y = a + b
# concatanate into a matrix
data_np = np.array((a,b)).T
# true labels
labels_np = np.vstack(y)

### normalize data

In [62]:
# print(data_np)
data_norm = data_np/np.max(data_np)
data_norm

array([[-1.42857143, -0.71428571],
       [ 0.57142857,  0.42857143],
       [ 0.42857143,  0.85714286],
       [ 0.85714286,  0.85714286],
       [ 0.        ,  0.85714286],
       [ 0.71428571, -0.85714286],
       [-0.42857143,  0.71428571],
       [ 0.71428571, -1.28571429],
       [ 0.85714286,  1.        ],
       [ 0.28571429,  0.71428571]])

## Create train/test group using data loader

In [106]:
# step 1 - convert to tensors
datat = torch.tensor(data_np).float()
labelt = torch.tensor(labels_np).float()

# step 2 - use scikit to split data
X_train, X_test, y_train, y_test = train_test_split(datat, labelt, test_size=0.2, random_state=101)

# convert to pytorch data sets
train_data = TensorDataset(X_train,y_train)
test_data = TensorDataset(X_test,y_test)

# translate to dataloader objects
batch_size = 2
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])
print(train_loader)
print(test_loader)

In [107]:
print(test_data.tensors[0])
next(iter(train_loader))

tensor([[6., 7.],
        [3., 6.]])


[tensor([[-3.,  5.],
         [ 2.,  5.]]),
 tensor([[2.],
         [7.]])]

In [145]:
# define and create the model
def createModel():
    
    class sumFunctionANN(nn.Module):
        # initialize
        def __init__(self):
            super().__init__()
            ## input layer.
            self.input = nn.Linear(2,2)
            ## hidden layer
            self.fc1 = nn.Linear(2,2)
            ## output layer
            self.output = nn.Linear(2,1)
            # create the loss function
            self.lossfunc = nn.MSELoss()
            # learning rate
            self.learningRate = .01
            # optimizer
            self.optimizer = torch.optim.SGD(self.parameters(),lr=learningRate)
        
        # forward pass
        def forward(self,x):
            # pass through input layer
            x = F.relu(self.input(x))
            # pass through hidden layer
            x = F.relu(self.fc1(x))
            # pass through output layer
            x = torch.sigmoid(self.output(x))
            return x
        
        # train the model
        def train(self,X,y):
            batchAcc  = []
            batchLoss = []
            
            # forward pass and loss
            yHat = self.forward(X)
            loss = self.lossfunc(yHat,y)
            
            # backprop
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            # loss from this batch
            batchLoss.append(loss.item())
            
            # compute accuracy
            matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
            matchesNumeric = matches.float()             # convert to numbers (0/1)
            accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
            batchAcc.append( accuracyPct )               # add to list of accuracies
            
            return batchAcc,batchLoss
            
            
    # create an instance of the model class
    model = sumFunctionANN()
    
    
    return model
        

In [148]:
# test the model with 1 batch
m = createModel()
m
# test with 1 instance of data
X ,y = next(iter(train_loader))
batchAcc,batchLoss = m.train(X,y)

print(batchLoss)
print(batchAcc)

[139.99896240234375]
[tensor(0.)]


# Function to train the model

In [14]:
def trainModel():
    # no of epochs
    numofepochs = 100
    # create a new model
    model, lossfunc, optimizer = createModel()
    # initialize losses
    losses = torch.zeros(numofepochs)
    trainAcc = []
    testAcc = []
    
    # loop over epochs
    for epochi in range(numofepochs):
        
        # loop over batches
        batchAcc = []
        batchLoss = []
        
        for X,y in train_loader:
            
            # forward pass
            yHat = model(X)
            loss = lossfunc(yHat,y)
            
            # back propogation
            
    

array([  6,  16, -10,   7,  -3,  -4, -10,  -4,  -1,   2])